# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1631937024592_0319'

In [3]:
pd.options.display.max_columns = 35

In [4]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831


In [5]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [6]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [7]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay


In [8]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [9]:
spark.sql("""select * from questionnaire """).printSchema()

root
 |-- questionnaireid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- status: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |  

In [10]:
spark.sql("""select * from demographics""").printSchema()

root
 |-- personid: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- dateofdeath: string (nullable = true)
 |-- gender: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- birthsex: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |

In [11]:
spark.sql("""select * from encounter""").printSchema()

root
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- reasonforvisit: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- financialclass: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullabl

In [12]:
spark.sql("""select * from medication""").printSchema()

root
 |-- medicationid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- intendeddispenser: string (nullable = true)
 |-- startdate: string (nullable = true)
 |-- intendedadministrator: string (nullable = true)
 |-- doseunit: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- stopdate: string (nullable = true)
 |-- category: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (n

In [13]:
statsDf = spark.sql(f"""

  SELECT 
      round(mean(length_of_stay),2) as mean_length_of_stay
     ,round(std(length_of_stay),2) as std_length_of_stay
     ,max(length_of_stay) as max_length_of_stay
  FROM (
    SELECT datediff(e.dischargeDate, e.serviceDate)+1 as length_of_stay

        FROM {db}.encounter e

        JOIN {db}.demographics d
            ON e.personId = d.personId
               and (d.deceased = false or d.deceased is null)
               
        JOIN {db}.condition c
            ON e.personId = c.personId
              and e.encounterId = c.encounterId
              and ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                    'CORONAVIRUS_COVID_19_POSITIVE_CLIN', 'CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN',
                    'CORONAVIRUS_COVID_19_TRAVEL_TO_AREA_AFFECTED_CLIN'), '{defaultcontext}' ) )  

        WHERE  ( HAS_ANY_CONCEPT_IN_CONTEXT(e.encounterTypes.type, array('INPATIENT_VISIT_ENC'), '{defaultcontext}' ) 
                  or HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('INPATIENT_VISIT_ENC'), '{defaultcontext}' ) )
            and e.dischargeDate is not null
        )
    WHERE length_of_stay > 0    


""")

In [14]:
mean_length_of_stay = statsDf.first().mean_length_of_stay
print(mean_length_of_stay)

7.5


In [15]:
std_length_of_stay = statsDf.first().std_length_of_stay
print(std_length_of_stay)

10.25


In [16]:
max_length_Of_stay = statsDf.first().max_length_of_stay

In [17]:
print(max_length_Of_stay)

4059


In [18]:
max_length_of_stay = mean_length_of_stay + (std_length_of_stay)*4

In [19]:
print(max_length_of_stay)

48.5


In [22]:
statsDf = spark.sql(f"""

  SELECT 
      round(mean(length_of_stay),2) as mean_length_of_stay
     ,round(std(length_of_stay),2) as std_length_of_stay
  FROM (
    SELECT datediff(e.dischargeDate, e.serviceDate)+1 as length_of_stay

        FROM {db}.encounter e

        JOIN {db}.demographics d
            ON e.personId = d.personId
               and (d.deceased = false or d.deceased is null)
               
        JOIN {db}.condition c
            ON e.personId = c.personId
              and e.encounterId = c.encounterId
              and ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                    'CORONAVIRUS_COVID_19_POSITIVE_CLIN', 'CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN',
                    'CORONAVIRUS_COVID_19_TRAVEL_TO_AREA_AFFECTED_CLIN'), '{defaultcontext}' ) )     
             
            

        WHERE  ( HAS_ANY_CONCEPT_IN_CONTEXT(e.encounterTypes.type, array('INPATIENT_VISIT_ENC'), '{defaultcontext}' ) 
                  or HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('INPATIENT_VISIT_ENC'), '{defaultcontext}' ) )
            and e.dischargeDate is not null
            
            
            and  ( exists (select * from medication m
                        where e.personId = m.personId
                          AND HAS_ANY_CONCEPT_IN_CONTEXT(m.drugCode,
                            array('SARS_COV2_COVID19_CVX_207_MED','SARS_COV2_COVID19_CVX_207_VAC',
                            'SARS_COV2_COVID19_CVX_208_MED','SARS_COV2_COVID19_CVX_208_VAC',
                            'SARS_COV2_COVID19_CVX_210_MED','SARS_COV2_COVID19_CVX_210_VAC',
                            'SARS_COV2_COVID19_CVX_212_MED','SARS_COV2_COVID19_CVX_212_VAC',
                            'SARS_COV2_COVID19_CVX_213_MED','SARS_COV2_COVID19_CVX_213_VAC',
                            'SARS_COV2_COVID19_CVX_500_MED','SARS_COV2_COVID19_CVX_500_VAC',
                            'SARS_COV2_COVID19_CVX_510_MED','SARS_COV2_COVID19_CVX_510_VAC',
                            'SARS_COV2_COVID19_CVX_511_MED','SARS_COV2_COVID19_CVX_511_VAC'), '{defaultcontext}' )
                          AND datediff(m.startdate, e.serviceDate) <=0  )
                   or exists (select * from procedure p
                        where e.personId = p.personId
                          AND HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('SARS_COV2_COVID19_SINGLE_DOSE_VACCINE_PROC','SARS_COV2_COVID19_VACCINE_PROC'),
                               '{defaultcontext}' )   
                             AND datediff(p.serviceStartDate, e.serviceDate) <=0  )  
                            
                 )
                 
                 
        )
    WHERE length_of_stay > 0    


""")

In [23]:
mean_length_of_stay_with_vaccine = statsDf.first().mean_length_of_stay
print(mean_length_of_stay_with_vaccine)

7.12


In [24]:
std_length_of_stay_with_vaccine = statsDf.first().std_length_of_stay
print(std_length_of_stay_with_vaccine)

5.0


In [25]:
statsDf = spark.sql(f"""

  SELECT 
      round(mean(length_of_stay),2) as mean_length_of_stay
     ,round(std(length_of_stay),2) as std_length_of_stay
  FROM (
    SELECT datediff(e.dischargeDate, e.serviceDate)+1 as length_of_stay

        FROM {db}.encounter e

        JOIN {db}.demographics d
            ON e.personId = d.personId
               and (d.deceased = false or d.deceased is null)
               
        JOIN {db}.condition c
            ON e.personId = c.personId
              and e.encounterId = c.encounterId
              and ( HAS_ANY_CONCEPT_IN_CONTEXT(c.conditionCode, array('CORONAVIRUS_COVID_19_DISEASE_CLIN',
                    'CORONAVIRUS_COVID_19_POSITIVE_CLIN', 'CORONAVIRUS_COVID_19_EXPOSURE_TO_CLIN',
                    'CORONAVIRUS_COVID_19_TRAVEL_TO_AREA_AFFECTED_CLIN'), '{defaultcontext}' ) )     
             
            

        WHERE  ( HAS_ANY_CONCEPT_IN_CONTEXT(e.encounterTypes.type, array('INPATIENT_VISIT_ENC'), '{defaultcontext}' ) 
                  or HAS_ANY_CONCEPT_IN_CONTEXT(e.type, array('INPATIENT_VISIT_ENC'), '{defaultcontext}' ) )
            and e.dischargeDate is not null
            
            
            and ( not exists (select * from medication m
                        where e.personId = m.personId
                          AND HAS_ANY_CONCEPT_IN_CONTEXT(m.drugCode,
                            array('SARS_COV2_COVID19_CVX_207_MED','SARS_COV2_COVID19_CVX_207_VAC',
                            'SARS_COV2_COVID19_CVX_208_MED','SARS_COV2_COVID19_CVX_208_VAC',
                            'SARS_COV2_COVID19_CVX_210_MED','SARS_COV2_COVID19_CVX_210_VAC',
                            'SARS_COV2_COVID19_CVX_212_MED','SARS_COV2_COVID19_CVX_212_VAC',
                            'SARS_COV2_COVID19_CVX_213_MED','SARS_COV2_COVID19_CVX_213_VAC',
                            'SARS_COV2_COVID19_CVX_500_MED','SARS_COV2_COVID19_CVX_500_VAC',
                            'SARS_COV2_COVID19_CVX_510_MED','SARS_COV2_COVID19_CVX_510_VAC',
                            'SARS_COV2_COVID19_CVX_511_MED','SARS_COV2_COVID19_CVX_511_VAC'), '{defaultcontext}' ) )
                   and not exists (select * from procedure p
                        where e.personId = p.personId
                          AND HAS_ANY_CONCEPT_IN_CONTEXT(p.procedureCode,
                            array('SARS_COV2_COVID19_SINGLE_DOSE_VACCINE_PROC','SARS_COV2_COVID19_VACCINE_PROC'),
                               '{defaultcontext}' )  )        
                            
                 )
        )
    WHERE length_of_stay > 0    


""")

In [26]:
mean_length_of_stay_without_vaccine = statsDf.first().mean_length_of_stay
print(mean_length_of_stay_without_vaccine)

7.5


In [27]:
std_length_of_stay_with_vaccine = statsDf.first().std_length_of_stay
print(std_length_of_stay_without_vaccine)

NameError: name 'std_length_of_stay_without_vaccine' is not defined

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
# generating multiple normal distributions
domain = np.linspace(0, max_length_of_stay, 100) # dividing the distance between 0 and max_length_of_stay into 100 points

means = [mean_length_of_stay, mean_length_of_stay_with_vaccine, mean_length_of_stay_without_vaccine]
std_values = [std_length_of_stay, std_length_of_stay_with_vaccine, std_length_of_stay_without_vaccine]
title_values = ["all", "with vaccination", "without vaccination"]

plt.figure(figsize=(16, 9))
for title, mu, std in zip(title_values, means, std_values):
    # pdf stands for Probability Density Function, which is the plot the probabilities of each range of values
    probabilities = norm.pdf(domain, mu, std)
    plt.plot(domain, probabilities, label=f"title={title}$\n$\mu={mu}$\n$\sigma={std}$\n")

plt.legend()
plt.xlabel("Value")
plt.ylabel("Probability")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data1 = np.random.normal(0, 1, 3000)
data2 = np.random.normal(-2.6, 1.8, 3000)
data3 = np.random.normal(2.4, 1.5, 3000)

fig, axe = plt.subplots(dpi=800)
axe.hist(data1, bins=40, density=True, histtype='stepfilled', alpha=0.3, label="mu=0,std=1")
axe.hist(data2, bins=40, density=True, histtype='stepfilled', alpha=0.3, label="mu=-2.6,std=1.8")
axe.hist(data3, bins=40, density=True, histtype='stepfilled', alpha=0.3, label="mu=2.4,std=1.5")
axe.legend()
fig.savefig("output/output.png")
plt.close(fig)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sigma = 1
mu = 0
fig, axe = plt.subplots(dpi=800)
data = np.random.normal(mu, sigma, 3000)
n, bins, _ = axe.hist(data, bins=40, density=True)
y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
     np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
axe.plot(bins, y, '--', color='r')
fig.savefig("output/output.png")
plt.close(fig)

In [ ]:
# Solution
import seaborn as sns
df = sns.load_dataset('iris')

plt.subplots(figsize=(7,6), dpi=100)
sns.distplot( df.loc[df.species=='setosa', "sepal_length"] , color="dodgerblue", label="Setosa")
sns.distplot( df.loc[df.species=='virginica', "sepal_length"] , color="orange", label="virginica")
sns.distplot( df.loc[df.species=='versicolor', "sepal_length"] , color="deeppink", label="versicolor")

plt.title('Iris Histogram')
plt.legend();